In [236]:
import pymongo
import json
import pandas as pd
import time
import re
import numpy as np
from gmplot import gmplot

In [92]:
client=pymongo.MongoClient()

In [93]:
client.list_database_names()

['admin', 'config', 'local', 'pollution', 'twitterdb']

In [94]:
db=client.pollution
db.collection_names()

['tweetcollection']

In [95]:
db.tweetcollection.find({"place":{"$ne": None}}).count() # with place tweets

24995

In [96]:
db.tweetcollection.find({"geo":{"$ne": None}}).count() # with exact location

1013

In [97]:
pollution_pattern="pollution"

In [98]:
category_0=["air #pollution","空气污染","#airpollution","वायु प्रदुषण","la pollution de l'air","la #pollution","air pollution","l'air #pollution"]

In [99]:
category_0

['air #pollution',
 '空气污染',
 '#airpollution',
 'वायु प्रदुषण',
 "la pollution de l'air",
 'la #pollution',
 'air pollution',
 "l'air #pollution"]

In [100]:
regx = re.compile(r'('+'|'.join(category_0)+')', re.IGNORECASE)
regx

re.compile(r"(air #pollution|空气污染|#airpollution|वायु प्रदुषण|la pollution de l'air|la #pollution|air pollution|l'air #pollution)",
re.IGNORECASE|re.UNICODE)

In [101]:
withlocation=list(db.tweetcollection.find({"geo":{"$ne": None},"text":{"$regex":pollution_pattern}}))

In [102]:
len(withlocation)

11

In [128]:
withlocation[1]

{'_id': 994691118179602432,
 'contributors': None,
 'coordinates': {'coordinates': [114.15893555, 22.35388613], 'type': 'Point'},
 'created_at': 'Thu May 10 21:30:17 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'dlvr.it/QSRjd7',
    'expanded_url': 'http://dlvr.it/QSRjd7',
    'indices': [94, 117],
    'url': 'https://t.co/fqKCIvjV1z'}],
  'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': {'coordinates': [22.35388613, 114.15893555], 'type': 'Point'},
 'id_str': '994691118179602432',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': {'attributes': {},
  'bounding_box': {'coordinates': [[[114.141298, 22.344083],
     [114.141298, 22.439185],
     [114.25441, 22.439185],
     [114.25441, 22.344083]]],
   'type': 'Polygon'},
  'country':

In [106]:
withplace=list(db.tweetcollection.find({"place":{"$ne": None}}))

In [108]:
withplace[0]

{'_id': 994256948102021121,
 'contributors': None,
 'coordinates': None,
 'created_at': 'Wed May 09 16:45:03 +0000 2018',
 'display_text_range': [0, 76],
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.twitter.com/xu2PMQKcrj',
    'expanded_url': 'https://twitter.com/ValleyGreenTeam/status/994256948102021121/photo/1',
    'id': 994256937708503041,
    'id_str': '994256937708503041',
    'indices': [77, 100],
    'media_url': 'http://pbs.twimg.com/media/DcxPasOVQAEqUPY.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DcxPasOVQAEqUPY.jpg',
    'sizes': {'large': {'h': 1536, 'resize': 'fit', 'w': 2048},
     'medium': {'h': 900, 'resize': 'fit', 'w': 1200},
     'small': {'h': 510, 'resize': 'fit', 'w': 680},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'type': 'photo',
    'url': 'https://t.co/xu2PMQKcrj'}],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 772369988,
    'id_str': '772369988',
    'indices': [0, 10],
    'name': 'Dibs',
    

In [221]:
def get_coordinate(this_tweet):
    '''input  dict  tweet
       output list [LONGITUDE LATITUDE]'''
    # There are geo-tagged 3 place store geoinformation 
    # User self-report place 
    # geo  1 Latitude 2 Longitude 
    # coordinates 1 LONGITUDE,2 Latitude 
    # coordinate box same as coordinates
    coordiantes=None
    try:
        coordinates=this_tweet['coordinates']['coordinates']
#         print('coordinates {}'.format(this_tweet['coordinates']['coordinates']))
    except Exception as e:
        pass
    
    try:
        # take average over coordinates box
        coordinates=list(np.array(this_tweet['place']['bounding_box']['coordinates'][0]).mean(axis=0))
    except Exception as e:
        pass

    return coordinates

In [252]:
def get_retweetid(tweet):
    try:
        return(tweet["retweeted_status"]["id"])
    except: 
        return None
    
def get_retweet_username(tweet):
    try:
        return(tweet["retweeted_status"]["user"]["name"])
    except: 
        return None
def get_retweet_followers_count(tweet):
    try:
        return(tweet["retweeted_status"]["user"]["followers_count"])
    except: 
        return None
def get_retweet_fa(tweet):
    try:
        return(tweet["retweeted_status"]["favorite_count"])
    except: 
        return None
def get_retweet_retw(tweet):
    try:
        return(tweet["retweeted_status"]["retweet_count"])
    except: 
        return None
def get_retweet_rep(tweet):
    try:
        return(tweet["retweeted_status"]["reply_count"])
    except: 
        return None
    
            
    
def jsontopandas(tweets_data):
    tweets=pd.DataFrame()
    tweets['id'] = list(map(lambda tweet: tweet['_id'], tweets_data))
    tweets['longitude']=list(map(lambda tweet:get_coordinate(tweet)[0], tweets_data))
    tweets['latitude']=list(map(lambda tweet: get_coordinate(tweet)[1], tweets_data))
    tweets['place_country']=list(map(lambda tweet: tweet['place']['country'] if tweet['place']!= None else None, tweets_data))
    tweets['place_full_name']=list(map(lambda tweet: tweet['place']['full_name'] if tweet['place']!= None else None, tweets_data))
    tweets['text']=list(map(lambda tweet: tweet['text'], tweets_data))
    tweets["user_id"]=list(map(lambda tweet:tweet["user"]["id"],tweets_data))
    tweets["user_name"]=list(map(lambda tweet:tweet["user"]["name"],tweets_data))
    tweets["user_followers"]=list(map(lambda tweet:tweet["user"]["followers_count"],tweets_data))
    tweets['user_loc']=list(map(lambda tweet: tweet['user']['location'], tweets_data))
    tweets['lang']=list(map(lambda tweet: tweet['lang'], tweets_data))
    tweets['created_at']=list(map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')), tweets_data))
    return tweets

In [253]:
thistweet=jsontopandas(withplace)

In [254]:
thistweet.head()

,id,longitude,latitude,place_country,place_full_name,text,user_id,user_name,user_followers,user_loc,lang,created_at
0,994256948102021121,-121.300425,37.980826,United States,"Stockton, CA",@DibsMyWay now presenting on reducing air poll...,1651496246,Valley Green Team,76,"Stockton, CA",en,2018-05-09 16:45:03
1,994266461790072832,-98.609315,29.496199,United States,"Leon Valley, TX",Air Pollution Project presentation board!,103292216,Dave Gannon,65,San Antonio,en,2018-05-09 17:22:51
2,994272672845631498,72.874244,19.174140,India,"Mumbai, India",@AnupriyaSPatel At least you have to visit शकत...,1614927878,Amit singh,113,"Mumbai, India",en,2018-05-09 17:47:32
3,994277435469316097,77.556422,13.075209,India,"Bengaluru, India",Do you paint? Or appreciate mesmerizing painti...,941583813536137216,BuzzOnEarth,182,India,en,2018-05-09 18:06:28
4,994288544368791554,-77.275953,18.115407,Jamaica,Jamaica,"Today on #TVJAllAngles - burning tyres, #NotTh...",73107660,Dionne JacksonMiller,44456,Jamaica,en,2018-05-09 18:50:36


In [234]:
thistweet.to_csv("../output/tweet_with_location.csv",encoding='utf-8')

In [235]:
this_tweet=pd.read_csv("../output/tweet_with_location.csv")
this_tweet.head()

,Unnamed: 0,id,tweet_longitude,tweet_latitude,place_country,place_full_name,text,user_id,user_name,user_followers,user_loc,lang,created_at
0,0,994256948102021121,-121.30042549999999,37.980826,United States,"Stockton, CA",@DibsMyWay now presenting on reducing air poll...,1.651496e+09,Valley Green Team,76.0,"Stockton, CA",en,2018-05-09 16:45:03
1,1,994266461790072832,-98.609315,29.496199,United States,"Leon Valley, TX",Air Pollution Project presentation board!,1.032922e+08,Dave Gannon,65.0,San Antonio,en,2018-05-09 17:22:51
2,2,994272672845631498,72.874244,19.174140,India,"Mumbai, India",@AnupriyaSPatel At least you have to visit शकत...,1.614928e+09,Amit singh,113.0,"Mumbai, India",en,2018-05-09 17:47:32
3,3,994277435469316097,77.5564225,13.075209,India,"Bengaluru, India",Do you paint? Or appreciate mesmerizing painti...,9.415838e+17,BuzzOnEarth,182.0,India,en,2018-05-09 18:06:28
4,4,994288544368791554,-77.27595299999999,18.115407,Jamaica,Jamaica,"Today on #TVJAllAngles - burning tyres, #NotTh...",7.310766e+07,Dionne JacksonMiller,44456.0,Jamaica,en,2018-05-09 18:50:36


In [256]:
thistweet.longitude

0       -121.300425
1        -98.609315
2         72.874244
3         77.556422
4        -77.275953
5       -113.492617
6        -43.441578
7        -73.968542
8        -74.184848
9          2.288745
10         2.468568
11       -58.442643
12       -79.420865
13      -119.782465
14      -119.319526
15       -73.997118
16       150.931975
17       -82.226701
18       -73.948775
19        84.133020
20        -4.241751
21         4.375364
22         9.372679
23        -0.297430
24        -2.366933
25        -1.272298
26        77.095086
27        -2.366933
28        77.498726
29        -0.350693
            ...    
24965     29.005223
24966     30.029373
24967     -3.997960
24968     -3.468006
24969     36.843477
24970     32.756773
24971     27.938921
24972     29.005223
24973     29.005223
24974   -100.076888
24975     29.005223
24976     29.005223
24977   -118.125196
24978     35.284880
24979    -83.804475
24980    -75.386461
24981     -0.561272
24982     29.005223
24983     29.120529


In [251]:
help(gmap.heatmap)

Help on method heatmap in module gmplot.gmplot:

heatmap(lats, lngs, threshold=10, radius=10, gradient=None, opacity=0.6, dissipating=True) method of gmplot.gmplot.GoogleMapPlotter instance
    :param lats: list of latitudes
    :param lngs: list of longitudes
    :param threshold:
    :param radius: The hardest param. Example (string):
    :return:



In [258]:
from gmplot import gmplot

# Place map
gmap = gmplot.GoogleMapPlotter(37.766956, -122.438481, 2)

# # Polygon
# golden_gate_park_lats, golden_gate_park_lons = zip(*[
#     (37.771269, -122.511015),
#     (37.773495, -122.464830),
#     (37.774797, -122.454538),
#     (37.771988, -122.454018),
#     (37.773646, -122.440979),
#     (37.772742, -122.440797),
#     (37.771096, -122.453889),
#     (37.768669, -122.453518),
#     (37.766227, -122.460213),
#     (37.764028, -122.510347),
#     (37.771269, -122.511015)
#     ])
# gmap.plot(golden_gate_park_lats, golden_gate_park_lons, 'cornflowerblue', edge_width=10)

# Scatter points
# top_attraction_lats, top_attraction_lons = zip(*[
#     (37.769901, -122.498331),
#     (37.768645, -122.475328),
#     (37.771478, -122.468677),
#     (37.769867, -122.466102),
#     (37.767187, -122.467496),
#     (37.770104, -122.470436)
#     ])
gmap.scatter(thistweet.latitude, thistweet.longitude, '#3B0B39', size=40, marker=False)

# # Marker
# hidden_gem_lat, hidden_gem_lon = 37.770776, -122.461689
# gmap.marker(hidden_gem_lat, hidden_gem_lon, 'cornflowerblue')

# Draw
gmap.draw("output/my_map2.html")

In [259]:
gmap.heatmap(thistweet.latitude, thistweet.longitude)
gmap.draw("output/my_map3.html")

In [22]:
with open("../output/withplace.json","a") as f:
    for tweet in withplace:
        f.write(json.dumps(tweet))